## Importando bibliotecas e definindo constantes de login e caminho de arquivo

In [ ]:
import psycopg2
import pandas as pd
import csv

NAME = 'projeto_mc536'
USER = 'postgres'
PASSWORD = 'hab1b@ot0p'
HOST = 'localhost'
PORT = '5432'

# Caminhos para os arquivos CSV utilizados
PROC_ENERGY_CSV = 'bancos_preprocessados/energia.csv'

## Funções para criação da conexão, fechamento do cursor e fechamento da conexão com o banco de dados

In [ ]:
def create_connection():
	"""Cria uma conexão com o banco de dados PostgreSQL."""
	try:
		conn = psycopg2.connect(
			dbname=NAME,
			user=USER,
			password=PASSWORD,
			host=HOST,
			port=PORT
		)
		return conn
	except Exception as e:
		print(f"Erro ao conectar ao banco de dados: {e}")
		return None
	
def close_cursor(cursor):
	"""Fecha o cursor."""
	if cursor:
		cursor.close()
		print("Cursor fechado.")

def end_connection(conn):
	"""Faz commit e encerra a conexão com o banco de dados."""
	if conn:
		conn.commit()
		conn.close()
		print("Conexão encerrada.")

## Função para uma query SQL utilizando a conexão fornecida, exibindo os resultados e salvando em um arquivo CSV

In [ ]:
def execute_query(conn, num, query):
	"""Executa uma consulta SQL e retorna os resultados."""
	cursor = conn.cursor()
	try:
		cursor.execute(query)
		results = cursor.fetchall()
		colnames = [desc[0] for desc in cursor.description]
		df = pd.DataFrame(results, columns=colnames)
		print(df.to_string(index=False))
		df.to_csv(f'resultados/resultado_query_{num}.csv', index=False)
		print("Query executada com sucesso.")
	except Exception as e:
		print(f"Erro ao executar consulta: {e}")
		return None
	finally:
		close_cursor(cursor)

## Função para criar o modelo físico do banco de dados

In [4]:
def create_tables(conn):
	"""Cria todas as tabelas seguindo o modelo relacional."""
	cursor = conn.cursor()
	try:
		cursor.execute("""
			-- This script was generated by the ERD tool in pgAdmin 4.
				-- Please log an issue at https://github.com/pgadmin-org/pgadmin4/issues/new/choose if you find any bugs, including reproduction steps.
				BEGIN;


				CREATE TABLE IF NOT EXISTS public.pais
				(
					codigo character varying(5) NOT NULL,
					nome character varying(50) NOT NULL,
					continente character varying(20) NOT NULL,
					PRIMARY KEY (codigo)
				);

				CREATE TABLE IF NOT EXISTS public.grupo
				(
					id_grupo integer NOT NULL,
					sigla character varying(10) NOT NULL,
					PRIMARY KEY (id_grupo)
				);

				CREATE TABLE IF NOT EXISTS public.pais_grupo
				(
					pais_codigo character varying(5) NOT NULL,
					grupo_id_grupo integer NOT NULL
				);

				CREATE TABLE IF NOT EXISTS public.area
				(
					id integer NOT NULL,
					tipo character varying(10) NOT NULL,
					PRIMARY KEY (id)
				);

				CREATE TABLE IF NOT EXISTS public.distribuicao_territorial
				(
					pais_codigo character varying(5) NOT NULL,
					area_id integer NOT NULL,
					ano integer NOT NULL,
					area_km2 double precision NOT NULL
				);

				CREATE TABLE IF NOT EXISTS public.pib
				(
					ano integer NOT NULL,
					codigo_pais character varying NOT NULL,
					valor_usd double precision NOT NULL,
					PRIMARY KEY (ano, codigo_pais)
				);

				CREATE TABLE IF NOT EXISTS public.demografia
				(
					area_id integer NOT NULL,
					pais_codigo character varying(5) NOT NULL,
					ano integer,
					num_habitantes integer
				);

				CREATE TABLE IF NOT EXISTS public.gas
				(
					id_gas integer NOT NULL,
					nome_gas character varying(100) NOT NULL,
					formula_gas character varying(10),
					PRIMARY KEY (id_gas)
				);

				CREATE TABLE IF NOT EXISTS public.emissao_gas
				(
					gas_id_gas integer NOT NULL,
					pais_codigo character varying(5) NOT NULL,
					ano integer NOT NULL,
					emissao_mtco2 double precision NOT NULL
				);

				CREATE TABLE IF NOT EXISTS public.energia
				(
					id_energia integer NOT NULL,
					tipo character varying(100) NOT NULL,
					PRIMARY KEY (id_energia)
				);

				CREATE TABLE IF NOT EXISTS public.interacao_energetica
				(
					id_interacao integer NOT NULL,
					nome character varying(100) NOT NULL,
					unidade character varying(10) NOT NULL,
					PRIMARY KEY (id_interacao)
				);

				CREATE TABLE IF NOT EXISTS public.relatorio_energetico
				(
					codigo_pais character varying(5) NOT NULL,
					id_interacao integer NOT NULL,
					id_energia integer NOT NULL,
					ano integer NOT NULL,
					valor double precision NOT NULL
				);

				ALTER TABLE IF EXISTS public.pais_grupo
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.pais_grupo
					ADD FOREIGN KEY (grupo_id_grupo)
					REFERENCES public.grupo (id_grupo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.distribuicao_territorial
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.distribuicao_territorial
					ADD FOREIGN KEY (area_id)
					REFERENCES public.area (id) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.pib
					ADD FOREIGN KEY (codigo_pais)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.demografia
					ADD FOREIGN KEY (area_id)
					REFERENCES public.area (id) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.demografia
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.emissao_gas
					ADD FOREIGN KEY (gas_id_gas)
					REFERENCES public.gas (id_gas) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.emissao_gas
					ADD FOREIGN KEY (pais_codigo)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.relatorio_energetico
					ADD FOREIGN KEY (codigo_pais)
					REFERENCES public.pais (codigo) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.relatorio_energetico
					ADD FOREIGN KEY (id_interacao)
					REFERENCES public.interacao_energetica (id_interacao) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;


				ALTER TABLE IF EXISTS public.relatorio_energetico
					ADD FOREIGN KEY (id_energia)
					REFERENCES public.energia (id_energia) MATCH SIMPLE
					ON UPDATE NO ACTION
					ON DELETE NO ACTION
					NOT VALID;

				END;
		""")
		print("Tabelas criadas com sucesso.")
	except Exception as e:
		print(f"Erro ao criar tabelas: {e}")
	finally:
		close_cursor(cursor)

## Função para verificar tabelas existentes no banco de dados

In [5]:
def check_existing_tables(conn):
	cursor = conn.cursor()
	try:
		cursor.execute("""
			SELECT tablename
			FROM pg_catalog.pg_tables
			WHERE schemaname != 'pg_catalog' AND schemaname != 'information_schema';
		""")
		tables = cursor.fetchall()
		if tables:
			print("Tabelas existentes:")
			for table in tables:
				print(table[0])
		else:
			print("Nenhuma tabela encontrada.")
	except Exception as e:
		print(f"Erro ao verificar tabelas existentes: {e}")
	finally:
		close_cursor(cursor)

## Funções para popular individualmente tabelas do banco de dados com dados dos arquivos CSVs selecionados

In [ ]:
def populate_country_table(conn):
	cursor = conn.cursor()
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho

			for line in lines:
				# Pega os dados usando pandas
				country = line[0]
				country_code = line[1]
				continent = line[3]
				try:
					cursor.execute("""
						INSERT INTO pais (codigo, nome, continente)
						VALUES (%s, %s, %s)
						ON CONFLICT (codigo) DO NOTHING;
					""", (country_code, country, continent))		
				except Exception as e:
					print(f"Erro ao inserir país {country_code}: {e}")
					continue
		print("Tabela de país populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de país: {e}")
	finally:
		close_cursor(cursor)

def populate_group_table(conn):
	cursor = conn.cursor()
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			id_grupo = 1
			header = next(lines)
			for grupo in header[5:10]:
				cursor.execute("""
					INSERT INTO grupo (id_grupo, sigla)
					VALUES (%s, %s)
					ON CONFLICT (id_grupo) DO NOTHING;
				""", (id_grupo, grupo))
				id_grupo += 1
		print("Tabela de grupo populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de grupo de {grupo}: {e}")
	finally:
		close_cursor(cursor)

def populate_country_group_table(conn):
	cursor = conn.cursor()
	last_code = ''
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			for line in lines:
				if line[1] != last_code:
					last_code = line[1]
					for i in range(5, 10):
						print(line[i])
						if line[i] == '1.0':
							country_code = line[1]
							group_id = i - 4
							cursor.execute("""
								INSERT INTO pais_grupo (pais_codigo, grupo_id_grupo)
								VALUES (%s, %s);
							""", (country_code, group_id))
		print("Tabela país_grupo populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de país e grupo: {e}")
	finally:
		close_cursor(cursor)

def populate_energy_interaction_table(conn):
	cursor = conn.cursor()
	added = []
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			id = 1
			for line in lines:
				# Pega os dados usando pandas
				category = line[10]
				unit = line[13]
				# Checa se a interação já existe
				if category not in added:
					added.append(category)
					try:
						cursor.execute("""
							INSERT INTO interacao_energetica (id_interacao, nome, unidade)
							VALUES (%s, %s, %s)
							ON CONFLICT (id_interacao) DO NOTHING;
						""", (id, category, unit))
						id += 1
					except Exception as e:
						print(f"Erro ao inserir interação {category}: {e}")
						continue
		print("Tabela de interação populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de interação: {e}")
	finally:
		close_cursor(cursor)

def populate_energy_table(conn):
	cursor = conn.cursor()
	added = []
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			id = 1
			for line in lines:
				# Pega os dados usando pandas
				source = line[12]
				# Checa se a interação já existe
				if source not in added:
					added.append(source)
					try:
						cursor.execute("""
							INSERT INTO energia (id_energia, tipo)
							VALUES (%s, %s)
							ON CONFLICT (id_energia) DO NOTHING;
						""", (id, source))
						id += 1
					except Exception as e:
						print(f"Erro ao inserir interação {source}: {e}")
						continue
		print("Tabela de energia populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de energia: {e}")
	finally:
		close_cursor(cursor)

def populate_energy_report_table(conn):
	cursor = conn.cursor()
	# cria um dicionario sources com id e nome das energias
	sources = {}
	try:
		cursor.execute("SELECT id_energia, tipo FROM energia;")
		rows = cursor.fetchall()
		for row in rows:
			sources[row[1]] = row[0]
	except Exception as e:
		print(f"Erro ao buscar fontes de energia: {e}")
		return
	# cria um dicionario categories com id e nome das interações
	categories = {}
	try:
		cursor.execute("SELECT id_interacao, nome FROM interacao_energetica;")
		rows = cursor.fetchall()
		for row in rows:
			categories[row[1]] = row[0]
	except Exception as e:
		print(f"Erro ao buscar categorias de energia: {e}")
		return
	try:
		with open(PROC_ENERGY_CSV, 'r') as f:
			lines = csv.reader(f)
			next(lines)  # Pula o cabeçalho
			for line in lines:
				# Pega os dados usando pandas
				country_code = line[1]
				source = line[12]
				category = line[10]
				year = int(line[2])
				try:
					value = float(line[14])
					cursor.execute("""
						INSERT INTO relatorio_energetico (codigo_pais, id_interacao, id_energia, ano, valor)
						VALUES (%s, %s, %s, %s, %s);
					""", (country_code, categories[category], sources[source], year, value))
				except Exception as e:
					print(f"Erro ao inserir relatório energético: {e}")
					continue
		print("Tabela de relatório energético populada com sucesso.")
	except Exception as e:
		print(f"Erro ao popular tabela de relatório energético: {e}")
	finally:
		close_cursor(cursor)

## Chamando funções para popular as tabelas do BD

In [ ]:
conn = create_connection()
populate = False
populate_country = False
populate_group = False
populate_country_group = False
populate_interaction = False
populate_energy = False
populate_energy_report = False

if conn:
		create_tables(conn)
		# check_existing_tables(conn)
		if populate:
			if PROC_ENERGY_CSV:
				if populate_country:
					populate_country_table(conn)
				if populate_group:
					populate_group_table(conn)
				if populate_country_group:
					populate_country_group_table(conn)
				if populate_interaction:
					populate_energy_interaction_table(conn)
				if populate_energy:
					populate_energy_table(conn)
				if populate_energy_report:
					populate_energy_report_table(conn)
			else:
				print(f"Arquivo {PROC_ENERGY_CSV} não encontrado.")
		end_connection(conn)

Tabelas criadas com sucesso.
Cursor fechado.
Conexão encerrada.


# Query 1: Porcentagem de energia renovável produzida por grupo econômico e ano
Detecta a porcentagem renovável da energia total gerada por grupo econômico a cada ano.

In [ ]:
query = """
SELECT RE.ANO,
		G.SIGLA,
		ROUND((SUM(RE.VALOR)*100 / T.TOTAL)::NUMERIC, 2) AS PORCENTAGEM_RENOVAVEL
FROM PAIS P
INNER JOIN PAIS_GRUPO PG
	ON P.CODIGO = PG.PAIS_CODIGO
INNER JOIN GRUPO G
	ON PG.GRUPO_ID_GRUPO = G.ID_GRUPO
INNER JOIN RELATORIO_ENERGETICO RE
	ON P.CODIGO = RE.CODIGO_PAIS
INNER JOIN INTERACAO_ENERGETICA IE
	ON RE.ID_INTERACAO = IE.ID_INTERACAO
INNER JOIN ENERGIA E
	ON RE.ID_ENERGIA = E.ID_ENERGIA
INNER JOIN (
	SELECT	RE.ANO,
			G.SIGLA,
			SUM(RE.VALOR) AS TOTAL
		FROM PAIS P
	INNER JOIN PAIS_GRUPO PG
		ON P.CODIGO = PG.PAIS_CODIGO
	INNER JOIN GRUPO G
		ON PG.GRUPO_ID_GRUPO = G.ID_GRUPO
	INNER JOIN RELATORIO_ENERGETICO RE
		ON P.CODIGO = RE.CODIGO_PAIS
	INNER JOIN INTERACAO_ENERGETICA IE
		ON RE.ID_INTERACAO = IE.ID_INTERACAO
	INNER JOIN ENERGIA E
		ON RE.ID_ENERGIA = E.ID_ENERGIA
	WHERE IE.NOME LIKE '%generation'
	GROUP BY RE.ANO, G.SIGLA
) T
	ON T.ANO = RE.ANO AND T.SIGLA = G.SIGLA 
WHERE E.TIPO != 'Coal'
	AND E.TIPO != 'Gas'
	AND E.TIPO != 'Nuclear'
	AND E.TIPO != 'Other Fossil'
	AND IE.NOME LIKE '%generation'
GROUP BY RE.ANO, G.SIGLA, IE.NOME, T.TOTAL, IE.UNIDADE
ORDER BY RE.ANO, G.SIGLA;
"""
conn = create_connection()
execute_query(conn, 1, query)
end_connection(conn)